In [28]:
import sys
sys.version
# Ref: https://machinelearningmastery.com/prepare-news-articles-text-summarization/

'3.7.1 (default, Dec 14 2018, 13:28:58) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

## Preprocessing

In [29]:
import os
import pandas as pd
import re
# Cleaning the texts
import nltk
import string
# from nltk import
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

### Import data

In [30]:
# Read in all story files
def ls_fullpath(dir):
    return [os.path.join(dir, f) for f in os.listdir(dir)]

In [31]:
#  Read in each text file
def read_file(fname):
    f = open(fname, "r")
    content = f.read()
    f.close()
    return content

In [32]:
# Parse story into story and highlight
def parse_text_hl(text):
    idx = text.find("@highlight")
    # split into story and hl
    story, hl = text[:idx], text[idx:].split("@highlight")
    # remove whitespace
    hl = [hltext.strip() for hltext in hl if len(hltext) > 0]
    return story, hl

In [33]:
# TEST
# idx = story.find("@highlight")
# # split into story and hl
# text, hl = story[:idx], story[idx:].split("@highlight")
# # remove whitespace
# hl = [hltext.strip() for hltext in hl if len(hltext) > 0]
# text[:idx]

In [34]:
# Load all files in the directory, read them, parse them, split them into story and hl
def get_stories(path, publication, src_type):
    stories = []
    for f in ls_fullpath(path):
        txt = read_file(f)
        story, hl = parse_text_hl(txt)
        stories.append({'story': story, 'summary': hl,'publication': publication, 'type': src_type})
    return stories

In [35]:
# Load all files in the directory, read them, parse them, split them into story and hl
# Allows selection of file subset 
# def get_stories(path, k, publication, src_type):
#     stories = []
#     for f in ls_fullpath(path)[:k]:
#         txt = read_file(f)
#         story, hl = parse_text_hl(txt)
#         stories.append({'story': story, 'highlights': hl,'publication': publication, 'type': src_type})
#     return stories

In [ ]:
# Function to cleaning up text
def text_cleaner(txt):
    clean_txt = []
    # For each line in text
    for l in txt:
        # Remove newspaper info
        l = re.sub("\\(CNN\\)|\\(CNN\\) \\-\\- ", "", l)
        # Convert to lowercase
        l = l.lower()
        # Remove punctuation
        # Split by whitespace
        tokenizer = RegexpTokenizer(r"[a-z0-9?.;!&']+")        
        l = tokenizer.tokenize(l)
        # l = l.split()
        clean_txt.append(" ".join(l))
    # Remove empty strings
    clean_txt = [c_txt for c_txt in clean_txt if len(c_txt) > 0]
    return clean_txt

## Preprocess data and save as a pickle

In [50]:
path = "/Users/dbm/Downloads/cnn_stories/stories"
pub = "cnn.com"
# path = "/Users/dbm/Downloads/dailymail_stories/stories"
# pub = "dailymail.com"
story_dict = get_stories(path, publication=pub, src_type="news")
len(story_dict)

92579

In [51]:
# Clean stories
for story in story_dict:
    story["story"] = text_cleaner(story["story"].split("\n"))
    story["summary"] = text_cleaner(story["summary"])

In [1]:
# Save cleaned data in pickle format
from pickle import dump
dump(story_dict, open("cnn_news_summary.pkl", "wb"))
# dump(story_dict, open("dailymail_news_summary.pkl", "wb"))

NameError: name 'story_dict' is not defined

In [2]:
# Load CNN
import pickle
cnn_stories = pickle.load(open('/Users/dbm/Documents/Insight S19/data/cnn_news_summary.pkl', 'rb'))
print('Loaded CNN Stories %d' % len(cnn_stories))

Loaded CNN Stories 92579


In [3]:
# Load dailymail
dailymail_stories = pickle.load(open('/Users/dbm/Documents/Insight S19/data/dailymail_news_summary.pkl', 'rb'))
print('Loaded dailymail Stories %d' % len(dailymail_stories))

Loaded dailymail Stories 219506


In [4]:
# Combine both datasets
master = cnn_stories + dailymail_stories
len(master)

312085

In [6]:
dump(master, open("cnn_dmail_news_summary.pkl", "wb"))

In [53]:
# Access loaded stories
story_dict[0]

{'story': ['at the start of a big week for the higgs boson the most sought after particle in all of physics scientists in illinois said monday that they had crept closer to proving that the particle exists but had been unable to reach a definitive conclusion.',
  "the scientists outlined their final analysis based on more than 10 years of research and 500 trillion particle collisions using the u.s. department of energy's fermilab tevatron collider near batavia illinois whose budgetary woes shut it down last year.",
  'what is the higgs boson and why is it important?',
  'their announcement came two days before researchers at the large hadron collider under the alps are due to unveil their latest results at an eagerly awaited seminar at the cern particle physics laboratory in geneva switzerland.',
  'our data strongly point toward the existence of the higgs boson rob roser a spokesman for one of two independent experiments at the tevatron said in a statement. but it will take results fr

In [1]:
# From each line
# # Remove newspaper info
# l_proc = re.sub("\\(CNN\\)|\\(CNN\\) \\-\\- ", "", l)

# # Convert to lowercase
# l_proc = l_proc.lower()

# # Remove punctuation
# # Split by whitespace

# tokenizer = RegexpTokenizer(r'[a-z]+')
# l_proc = tokenizer.tokenize(l_proc)

# # Split the line by whitespace
# # l = l.split()

# len(l_proc)

# l_proc
# # if not idx:
# #     l =
# #     l = l[idx + len("(CNN)|(CNN) -- "):]

In [2]:
from newsroom import jsonl

# Read entire file:

with jsonl.open("/Users/dbm/Downloads/thin/train.data", gzip = True) as train_file:
    train = train_file.read()

# Read file entry by entry:

with jsonl.open("/Users/dbm/Downloads/thin/train.data", gzip = True) as train_file:
    for entry in train_file:
        print(entry["summary"], entry["text"])


In [ ]:
import random

from newsroom import jsonl
from newsroom.analyze import Fragments

with jsonl.open("/Users/dbm/Downloads/thin/train.data", gzip = True) as train_file:
    train = train_file.read()

# Compute stats on random training example:

entry = random.choice(train)
summary, text = train[0]["summary"], train[0]["text"]
fragments = Fragments(summary, text)

# Print paper metrics:

print("Coverage:",    fragments.coverage())
print("Density:",     fragments.density())
print("Compression:", fragments.compression())

# Extractive fragments oracle:

print("List of extractive fragments:")
print(fragments.strings())